In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

## Read in Player_By_Game Data from Scraper

In [2]:
# Read in all the partial data pieces

df1 = pd.read_csv('../data/offset_0-200000.csv')
df2 = pd.read_csv('../data/offset_200000-300000.csv')
df3 = pd.read_csv('../data/offset_300000-475000.csv')
df4 = pd.read_csv('../data/offset_475000-550000.csv')
df5 = pd.read_csv('../data/offset_550000-600000.csv')
df6 = pd.read_csv('../data/offset_600000-725000.csv')
df7 = pd.read_csv('../data/offset_725000-909000.csv')

In [3]:
# Aggregate all the segmented data into one dataframe

df = pd.concat([df1, df2, df3, df4, df5, df6, df7])

In [4]:
# Check starting shape

df.shape

(909700, 31)

In [5]:
df.head()

Unnamed: 0           player pos   date_game           school_id  \
0           0     Nate Wolters   G  2013-02-07  South Dakota State   
1           1       Mike James   G  2011-01-04               Lamar   
2           2   Marshon Brooks   G  2011-02-23          Providence   
3           3  Jimmer Fredette   G  2011-03-11       Brigham Young   
4           4    Markus Howard   G  2018-01-03           Marquette   

       opp_id game_result   gs  mp  fg     ...      orb  drb  trb  ast  stl  \
0        IPFW           W  1.0  40  17     ...        2    2    4    3    1   
1         NaN           W  0.0  28  18     ...        1    4    5    3    1   
2  Notre Dame           L  1.0  40  20     ...        2    3    5    4    1   
3  New Mexico           W  1.0  40  22     ...        0    2    2    4    1   
4  Providence           W  1.0  44  17     ...        0    1    1    2    2   

   blk  tov  pf  pts  game_score  
0    1    3   0   53        42.6  
1    0    1   4   52        36.3  
2    0    1   4   52        42.3  
3    0    2   2   52        36.5  
4    0    2   1   52        39.8  

[5 rows x 31 columns]

## EDA And Cleaning of Player_By_Game Data

In [6]:
df.columns

Index(['Unnamed: 0', 'player', 'pos', 'date_game', 'school_id', 'opp_id',
       'game_result', 'gs', 'mp', 'fg', 'fga', 'fg_pct', 'fg2', 'fg2a',
       'fg2_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'game_score'],
      dtype='object')

In [7]:
df.isnull().sum()

Unnamed: 0          0
player              0
pos                91
date_game           0
school_id           0
opp_id          38014
game_result         0
gs               1558
mp                  0
fg                  0
fga                 0
fg_pct         107301
fg2                 0
fg2a                0
fg2_pct        173922
fg3                 0
fg3a                0
fg3_pct        384069
ft                  0
fta                 0
ft_pct         415522
orb                 0
drb                 0
trb                 0
ast                 0
stl                 0
blk                 0
tov                 0
pf                  0
pts                 0
game_score          0
dtype: int64

In [8]:
# Drop these columns because they are derived explicity from existing features
df.drop(['fg_pct', 'fg2_pct', 'fg3_pct', 'ft_pct', 'trb'], axis=1, inplace=True)

In [9]:
# Drop erroneous column
df.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
# If we don't know whether they started or not, chances are they did not. We'll set np.nan 
# of 'gs' to 0.
df['gs'] = df['gs'].fillna(value=0)

In [11]:
# Check and drop duplicated rows, probably resulting from scraping overlap
df.duplicated().sum()

1526

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.isnull().sum()

player             0
pos               90
date_game          0
school_id          0
opp_id         37961
game_result        0
gs                 0
mp                 0
fg                 0
fga                0
fg2                0
fg2a               0
fg3                0
fg3a               0
ft                 0
fta                0
orb                0
drb                0
ast                0
stl                0
blk                0
tov                0
pf                 0
pts                0
game_score         0
dtype: int64

In [14]:
# We will be bringing in a lot of opponent data as features, so not knowing the opp_id is 
# useless to us. So let's drop the np.nan values in opp_id

df.dropna(inplace=True)

In [15]:
df.isnull().sum()

player         0
pos            0
date_game      0
school_id      0
opp_id         0
game_result    0
gs             0
mp             0
fg             0
fga            0
fg2            0
fg2a           0
fg3            0
fg3a           0
ft             0
fta            0
orb            0
drb            0
ast            0
stl            0
blk            0
tov            0
pf             0
pts            0
game_score     0
dtype: int64

In [16]:
# Check resulting shape. We still have 96% of our original rows. Pretty good
df.shape

(870131, 25)

In [17]:
# Convert date_game to datetime type

df['date_game'] = pd.to_datetime(df['date_game'])

In [18]:
# Create a year and month feature

df['year'] = df['date_game'].map(lambda x: x.year)
df['month'] = df['date_game'].map(lambda x: x.month)

In [19]:
# Get an idea of timeframe 

print(df['date_game'].min())
print(df['date_game'].max())

2010-11-08 00:00:00
2018-03-07 00:00:00


In [21]:
# Make a 'season' feature, where the year it ended is defined as the season

season_list = []
for i in df['date_game']:
    if i.month == 11 or i.month == 12:
        season_list.append(i.year + 1)
    else:
        season_list.append(i.year)

In [22]:
df['season'] = season_list

In [23]:
# Codify Win/Loss with 1/0
df['W'] = df['game_result'].map(lambda x: 1 if x == 'W' else 0)

In [24]:
# Dummify position
#df = pd.concat([df, pd.get_dummies(df['pos'])], axis=1)

In [25]:
# Drop game_result: We have one-hot encoded
df.drop('game_result', axis=1, inplace=True)

In [26]:
df.isnull().sum()

player        0
pos           0
date_game     0
school_id     0
opp_id        0
gs            0
mp            0
fg            0
fga           0
fg2           0
fg2a          0
fg3           0
fg3a          0
ft            0
fta           0
orb           0
drb           0
ast           0
stl           0
blk           0
tov           0
pf            0
pts           0
game_score    0
year          0
month         0
season        0
W             0
dtype: int64

In [27]:
df.dtypes

player                object
pos                   object
date_game     datetime64[ns]
school_id             object
opp_id                object
gs                   float64
mp                     int64
fg                     int64
fga                    int64
fg2                    int64
fg2a                   int64
fg3                    int64
fg3a                   int64
ft                     int64
fta                    int64
orb                    int64
drb                    int64
ast                    int64
stl                    int64
blk                    int64
tov                    int64
pf                     int64
pts                    int64
game_score           float64
year                   int64
month                  int64
season                 int64
W                      int64
dtype: object

## Quick Scrape for some School Data

In [31]:
stat_year = 2010
col_list = ['School', 'WinLossPct', 'SRS', 'SOS', 'Year']
school_df = pd.DataFrame(columns=col_list)

for i in range(2010,2019, 1):
    
    schools = []
    winlosspct = []
    srs = []
    sos = []
    stat_year = i
    
    this_df = pd.DataFrame(columns=col_list)
    
    URL = 'https://www.sports-reference.com/cbb/seasons/'+str(stat_year)+'-school-stats.html'
    response = requests.get(URL)
    print(response.status_code)
    soup = BeautifulSoup(response.text, 'lxml')
     
    td_schools = soup.find_all('td', {'data-stat':'school_name'})
    for element in td_schools:
        td_ref = element.find('a')
        try:
            schools.append(td_ref.text)
        except:
            schools.append(np.nan)
            
    td_winloss = soup.find_all('td', {'data-stat':'win_loss_pct'})
    for element in td_winloss:
        try:
            winlosspct.append(element.text)
        except:
            winlosspct.append(np.nan)
            
    td_srs = soup.find_all('td', {'data-stat':'srs'})
    for element in td_srs:
        try:
            srs.append(element.text)
        except:
            srs.append(np.nan)

    td_sos = soup.find_all('td', {'data-stat':'sos'})
    for element in td_sos:
        try:
            sos.append(element.text)
        except:
            sos.append(np.nan)
            
    this_df['School'] = schools
    this_df['WinLossPct'] = winlosspct
    this_df['SRS'] = srs
    this_df['SOS'] = sos
    this_df['Year'] = stat_year
    
    school_df = pd.concat([school_df, this_df])

200
200
200
200
200
200
200
200
200


In [32]:
school_df.isnull().sum()

School        0
WinLossPct    0
SRS           0
SOS           0
Year          0
dtype: int64

In [33]:
school_df.head()

School WinLossPct     SRS     SOS  Year
0           Air Force       .323   -4.90    3.13  2010
1               Akron       .686    2.82   -1.50  2010
2         Alabama A&M       .407  -20.19  -13.71  2010
3  Alabama-Birmingham       .735    9.46    2.90  2010
4       Alabama State       .516  -14.41  -12.02  2010

In [34]:
school_df.to_csv('../data/schools.csv')

## Engineer Defender Features

In [35]:
school_list = school_df['School']

In [36]:
game_dates = df['date_game'].unique()

In [37]:
# TEST!!!
game_dates = '2010-11-08'

In [52]:
for m in game_dates:
    for n in school_list:    
        date_mask = m
        school_mask = n

        df_myteam = df[(df['date_game']==date_mask) & (df['school_id']==school_mask)]
        df_myteam = df_myteam.sort_values(['gs', 'mp'], ascending=False)

        opp_school = list(df_myteam['opp_id'])[0]

        df_opp = df[(df['date_game']==date_mask) & (df['school_id']==opp_school)]
        df_opp = df_opp.sort_values(['gs', 'mp'], ascending=False)

        largest = min([len(df_opp), len(df_myteam)])

        df_myteam = df_myteam[0:largest]
        df_opp = df_opp[0:largest]

        my_team_index = df_myteam.index
        matchup_index = []

        opp_pos_list = df_opp['pos']

        for i in df_myteam['pos']:
            if i == 'PG':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PG':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SG':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'G':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break

            if i == 'SG':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SG':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PG':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'G':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break  

            if i == 'SF':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'F':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break

            if i == 'PF':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'C':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'F':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break

            if i == 'C':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'C':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break

            if i == 'G':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PG':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SG':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'G':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break

            if i == 'F':
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'PF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'SF':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
                for j in df_opp['pos'].index:
                    if df_opp['pos'][j] == 'F':
                        matchup_index.append(j)
                        df_opp['pos'].drop(j, inplace=True)
                        break
            try:
                matchup_index.append(df_opp['pos'].index[0])
                df_opp['pos'].drop(df_opp['pos'].index[0], inplace=True)
            except:
                pass
        df.merge()
        # Do a thing that appends select cols to the right


# Modeling

In [141]:
# Come back to set X to drop just pts and 'Unnamed:0'

X = df.drop(['pts'], axis=1)
y = df['pts']

In [143]:
X.columns

Index(['gs', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'game_score', 'year',
       'month', 'season', 'W', 'C', 'F', 'G', 'PF', 'PG', 'SF', 'SG'],
      dtype='object')

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [116]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(653562, 30)
(653562,)
(217855, 30)
(217855,)


In [117]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [119]:
cross_val_score(linreg, X_train, y_train)

array([1., 1., 1.])